# Poromt Engineering PreWrok

## Import required libraries

In [1]:
import pandas as pd
import os
import json
from dotenv import load_dotenv
import openai

# Load the environment variables
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base = os.getenv('openai_base_url')


## Define the completion function and import the data

In [2]:
# define the function to get the completions
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  
    )
    return response.choices[0].message["content"]

# Load the proposals file
df_proposals = pd.read_excel("Proposals_File.xlsx")

In [3]:
# Example of the first proposals
df_proposals.head()

,Proposal_Name,Proposal_Text
0,Proposal_001,Data analysis for the years 2019 to 2021 was p...
1,Proposal_002,Data from 2015 and 2016 were used to model the...
2,Proposal_003,Expecting growth in 2022 and 2023 based on 202...
3,Proposal_004,"Historical figures from 2016, 2017, and 2018 w..."
4,Proposal_005,"In the period 2021-2023, we saw a decline in t..."


## Trying with one example

In [4]:
# get one of the proposals
proposal_text = df_proposals["Proposal_Text"][0]
print(f'--- The proposal text is:\n{proposal_text}')
# define the prompt
text = """Extract the years from the body of text delimeted by triple backticks into a python dictionary
 in the format of {"year":[year1, year2, year3, ...]}, where the years are integers"""

# construct the prompt
prompt = f'{text}```{proposal_text}```'
print(f'\n--- The constructed prompt is:\n{prompt}')

# get the response from the model
response = get_completion(prompt)
print(f'\n--- The model response to the promt is:\n{response}')

# convert the response to a dictionary
response_dict = json.loads(response)
print(f'\n--- The response as a dictionary is:\n{response_dict}')

--- The proposal text is:
Data analysis for the years 2019 to 2021 was performed.

--- The constructed prompt is:
Extract the years from the body of text delimeted by triple backticks into a python dictionary
 in the format of {"year":[year1, year2, year3, ...]}, where the years are integers```Data analysis for the years 2019 to 2021 was performed.```

--- The model response to the promt is:
{"year": [2019, 2020, 2021]}

--- The response as a dictionary is:
{'year': [2019, 2020, 2021]}


## Automate the extraction with a lambda

In [5]:
# using lambda function to apply the function to the whole dataframe row by row and storing the result in a new column named Relevant_Years
df_proposals['Relevant_Years'] = df_proposals["Proposal_Text"].apply(
    lambda x: json.loads(get_completion(f'{text}```{x}```')))
# example of the first proposals
df_proposals.head()

,Proposal_Name,Proposal_Text,Relevant_Years
0,Proposal_001,Data analysis for the years 2019 to 2021 was p...,"{'year': [2019, 2020, 2021]}"
1,Proposal_002,Data from 2015 and 2016 were used to model the...,"{'year': [2015, 2016, 2017]}"
2,Proposal_003,Expecting growth in 2022 and 2023 based on 202...,"{'year': [2022, 2023]}"
3,Proposal_004,"Historical figures from 2016, 2017, and 2018 w...","{'year': [2016, 2017, 2018]}"
4,Proposal_005,"In the period 2021-2023, we saw a decline in t...","{'year': [2021, 2022, 2023]}"


In [6]:
# get the relevant years list from the first proposal
df_proposals["Relevant_Years"][0]['year']


[2019, 2020, 2021]

In [7]:
# save the dataframe to a new excel file
df_proposals.to_excel("Proposals_File_With_Years.xlsx", index=False)